In [1]:
import os
import requests
import gzip
import shutil
import sqlite3
import pandas as pd

In [2]:
import os
import requests
import gzip
import shutil
import sqlite3
import pandas as pd

# URL to download the datasets
url = "https://datasets.imdbws.com/"

# List of files to download
files = [
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz",
    "name.basics.tsv.gz"
]

# Directory to save files
directory = "imdb_cache"
if not os.path.exists(directory):
    os.makedirs(directory)

# Function to download and extract a file
def download_extract_file(file_name):
    # Check if file exists locally. If it does, don't download.
    if os.path.exists(os.path.join(directory, file_name[:-3])):
        print(f"File {file_name[:-3]} already exists locally. Skipping download.")
        return
    
    # If it doesn't exist, download and extract the file
    response = requests.get(url + file_name, stream=True)
    with open(os.path.join(directory, file_name), 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    with gzip.open(os.path.join(directory, file_name), 'rb') as f_in:
        with open(os.path.join(directory, file_name[:-3]), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(os.path.join(directory, file_name))  # remove the gzipped file

# Download and extract all files
for file in files:
    print(f"Processing {file}...")
    download_extract_file(file)

# Check if the SQLite database already exists. If it does, skip the remaining operations.
if os.path.exists('imdb.db'):
    print("imdb.db already exists. Skipping data loading.")
else:
    # Connect to SQLite database
    conn = sqlite3.connect('imdb.db')

    # Read each TSV file into a pandas DataFrame and write to the database
    for file in files:
        print(f"Loading {file} into database...")
        df = pd.read_csv(os.path.join(directory, file[:-3]), delimiter='\t', low_memory=False)
        
        # Check if 'titleId', 'tconst', or 'nconst' exists in the dataframe and set it as the index
        potential_indices = ['titleId', 'tconst', 'nconst']
        for index in potential_indices:
            if index in df.columns:
                df.set_index(index, inplace=True)
                break

        # Replace '.' with '_' in table names and remove '.tsv' from the end
        table_name = file.replace('.', '_')[:-7]
        df.to_sql(table_name, conn, if_exists='replace')

    # Close the connection
    conn.close()


Processing title.akas.tsv.gz...
File title.akas.tsv already exists locally. Skipping download.
Processing title.basics.tsv.gz...
File title.basics.tsv already exists locally. Skipping download.
Processing title.crew.tsv.gz...
File title.crew.tsv already exists locally. Skipping download.
Processing title.episode.tsv.gz...
File title.episode.tsv already exists locally. Skipping download.
Processing title.principals.tsv.gz...
File title.principals.tsv already exists locally. Skipping download.
Processing title.ratings.tsv.gz...
File title.ratings.tsv already exists locally. Skipping download.
Processing name.basics.tsv.gz...
File name.basics.tsv already exists locally. Skipping download.
imdb.db already exists. Skipping data loading.


In [3]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# Create a cursor
cur = conn.cursor()

# Execute a query to get all table names
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cur.fetchall()

# Print each table name
for table in tables:
    print(table[0])

# Close the connection
conn.close()


title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
name_basics


In [4]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "title_akas",
    "title_basics",
    "title_crew",
    "title_episode",
    "title_principals",
    "title_ratings",
    "name_basics"
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} LIMIT 5", conn)
    print(df)
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_akas:
     titleId  ordering                      title region language   
0  tt0000001         1                 Карменсіта     UA       \N  \
1  tt0000001         2                 Carmencita     DE       \N   
2  tt0000001         3  Carmencita - spanyol tánc     HU       \N   
3  tt0000001         4                 Καρμενσίτα     GR       \N   
4  tt0000001         5                 Карменсита     RU       \N   

         types     attributes isOriginalTitle  
0  imdbDisplay             \N               0  
1           \N  literal title               0  
2  imdbDisplay             \N               0  
3  imdbDisplay             \N               0  
4  imdbDisplay             \N               0  


Showing first few rows of table title_basics:
      tconst titleType            primaryTitle           originalTitle   
0  tt0000001     short              Carmencita              Carmencita  \
1  tt0000002     short  Le clown et ses chiens  Le clown 

In [5]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "title_akas",
    "title_basics",
    "title_crew",
    "title_episode",
    "title_principals",
    "title_ratings",
    "name_basics"
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} LIMIT 5", conn)
    pd.set_option('display.max_colwidth', None)  # Set the maximum width of each column
    print(df.to_string(index=False))  # Print the dataframe without the index column
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_akas:
  titleId  ordering                     title region language       types    attributes isOriginalTitle
tt0000001         1                Карменсіта     UA       \N imdbDisplay            \N               0
tt0000001         2                Carmencita     DE       \N          \N literal title               0
tt0000001         3 Carmencita - spanyol tánc     HU       \N imdbDisplay            \N               0
tt0000001         4                Καρμενσίτα     GR       \N imdbDisplay            \N               0
tt0000001         5                Карменсита     RU       \N imdbDisplay            \N               0


Showing first few rows of table title_basics:
   tconst titleType           primaryTitle          originalTitle isAdult startYear endYear runtimeMinutes                   genres
tt0000001     short             Carmencita             Carmencita       0      1894      \N              1        Documentary,Short
tt0000002     short 

In [7]:
# Verify ability to retrieve years of movies
# result = valid

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "title_basics"
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} WHERE tconst = 'tt0018037'", conn)
    print(df)
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_basics:
      tconst titleType     primaryTitle    originalTitle isAdult startYear   
0  tt0018037     movie  The Jazz Singer  The Jazz Singer       0      1927  \

  endYear runtimeMinutes               genres  
0      \N             88  Drama,Music,Musical  




In [10]:
# Verify ability to retrieve years of movies
# result = valid

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "name_basics",
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} WHERE nconst = 'nm7613653'", conn)
    print(df)
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table name_basics:
      nconst    primaryName birthYear deathYear primaryProfession   
0  nm7613653  Lisha Wheeler        \N        \N           actress  \

                            knownForTitles  
0  tt4419214,tt8851668,tt6110648,tt3560084  




# Misc Queries

In [6]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "title_akas",
    "title_basics",
    "title_crew",
    "title_episode",
    "title_principals",
    "title_ratings",
    "name_basics"
]

# Create an empty dictionary to store all schemas
schemas = {}

# Query and store the schema of each table
for table in tables:
    df = pd.read_sql_query(f"PRAGMA table_info({table})", conn)
    schemas[table] = df

# Close the connection
conn.close()

# Now the 'schemas' dictionary contains the schema of each table as a DataFrame
for table, schema in schemas.items():
    print(f"Schema of {table}:")
    print(schema)
    print("\n")


Schema of title_akas:
   cid             name     type  notnull dflt_value  pk
0    0          titleId     TEXT        0       None   0
1    1         ordering  INTEGER        0       None   0
2    2            title     TEXT        0       None   0
3    3           region     TEXT        0       None   0
4    4         language     TEXT        0       None   0
5    5            types     TEXT        0       None   0
6    6       attributes     TEXT        0       None   0
7    7  isOriginalTitle     TEXT        0       None   0


Schema of title_basics:
   cid            name  type  notnull dflt_value  pk
0    0          tconst  TEXT        0       None   0
1    1       titleType  TEXT        0       None   0
2    2    primaryTitle  TEXT        0       None   0
3    3   originalTitle  TEXT        0       None   0
4    4         isAdult  TEXT        0       None   0
5    5       startYear  TEXT        0       None   0
6    6         endYear  TEXT        0       None   0
7    7  runtime

In [7]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "name_basics"
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} WHERE primaryName = 'Fred Astaire'", conn)
    print(df)
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table name_basics:


       nconst   primaryName birthYear deathYear   
0   nm0000001  Fred Astaire      1899      1987  \
1  nm12584561  Fred Astaire        \N        \N   

                primaryProfession                           knownForTitles  
0  soundtrack,actor,miscellaneous  tt0053137,tt0050419,tt0031983,tt0072308  
1                            None                                       \N  




In [8]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# List of tables in the database
tables = [
    "name_basics"
]

# Query and print the first few rows of each table
for table in tables:
    print(f"Showing first few rows of table {table}:")
    df = pd.read_sql_query(f"SELECT * from {table} WHERE nconst = 'nm0680983'", conn)
    print(df)
    print("\n")

# Close the connection
conn.close()


Showing first few rows of table name_basics:
      nconst  primaryName birthYear deathYear          primaryProfession   
0  nm0680983  Elliot Page      1987        \N  actor,producer,soundtrack  \

                            knownForTitles  
0  tt1375666,tt1877832,tt0424136,tt0467406  




In [9]:
# # Connect to SQLite database
# conn = sqlite3.connect('imdb.db')

# # Create a cursor
# cursor = conn.cursor()

# # Perform the JOIN operations
# cursor.execute('''
#     CREATE VIEW IF NOT EXISTS combined AS
#     SELECT 
#         basics.tconst,
#         basics.titleType,
#         basics.primaryTitle,
#         basics.originalTitle,
#         basics.isAdult,
#         basics.startYear,
#         basics.endYear,
#         basics.runtimeMinutes,
#         basics.genres,
#         crew.directors,
#         crew.writers,
#         ratings.averageRating,
#         ratings.numVotes,
#         akas.title,
#         akas.region,
#         akas.language,
#         akas.types,
#         akas.attributes,
#         akas.isOriginalTitle,
#         principals.ordering,
#         principals.nconst,
#         principals.category,
#         principals.job,
#         principals.characters,
#         names.primaryName,
#         names.birthYear,
#         names.deathYear,
#         names.primaryProfession,
#         names.knownForTitles
#     FROM title_basics as basics
#     LEFT JOIN title_crew as crew ON basics.tconst = crew.tconst
#     LEFT JOIN title_ratings as ratings ON basics.tconst = ratings.tconst
#     LEFT JOIN title_akas as akas ON basics.tconst = akas.titleId
#     LEFT JOIN title_principals as principals ON basics.tconst = principals.tconst
#     LEFT JOIN name_basics as names ON principals.nconst = names.nconst
# ''')

# # Query all movies made in the US
# cursor.execute('''
#     SELECT * FROM combined WHERE region = 'US'
# ''')

# # Fetch the results
# results = cursor.fetchall()

# # Print the results
# for row in results:
#     print(row)

# # Close the connection
# conn.close()


In [10]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# Query and print the first few rows of each table
print(f"Showing first few rows of table title_akas:")
df = pd.read_sql_query(f"SELECT * from title_akas WHERE region  = 'US'", conn)
print(df)
print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_akas:
           titleId  ordering                                      title   
0        tt0000001         6                                 Carmencita  \
1        tt0000002         7                     The Clown and His Dogs   
2        tt0000005        10                           Blacksmith Scene   
3        tt0000005         1                        Blacksmithing Scene   
4        tt0000005         6                        Blacksmith Scene #1   
...            ...       ...                                        ...   
1449999  tt9916560         1  March of Dimes Presents: Once Upon a Dime   
1450000  tt9916620         1                          The Copeland Case   
1450001  tt9916702         1              Loving London: The Playground   
1450002  tt9916756         1                   Pretty Pretty Black Girl   
1450003  tt9916764         1                                         38   

        region language        types             attrib

In [11]:

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# Query and print the first few rows of each table
print(f"Showing first few rows of table title_akas:")
                        # SELECT *, MAX(tb.ordering) \
df = pd.read_sql_query(f" \
                        SELECT * \
                        FROM title_akas ta \
                            LEFT JOIN title_basics tb \
                                ON ta.titleId = tb.tconst \
                        WHERE ta.region  = 'US'\
                       ", conn)
print(df)
print("\n")

Showing first few rows of table title_akas:
           titleId  ordering                                      title   
0        tt0000001         6                                 Carmencita  \
1        tt0000002         7                     The Clown and His Dogs   
2        tt0000005        10                           Blacksmith Scene   
3        tt0000005         1                        Blacksmithing Scene   
4        tt0000005         6                        Blacksmith Scene #1   
...            ...       ...                                        ...   
1449999  tt9916560         1  March of Dimes Presents: Once Upon a Dime   
1450000  tt9916620         1                          The Copeland Case   
1450001  tt9916702         1              Loving London: The Playground   
1450002  tt9916756         1                   Pretty Pretty Black Girl   
1450003  tt9916764         1                                         38   

        region language        types             attrib

In [12]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# Query and print the first few rows of each table
print(f"Showing first few rows of table title_crew:")
df = pd.read_sql_query(f"SELECT * from title_crew WHERE tconst  = 'tt4154796'", conn)
print(df)
print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_crew:
      tconst            directors   
0  tt4154796  nm0751577,nm0751648  \

                                                                                                                   writers  
0  nm1321655,nm1321656,nm0498278,nm0456158,nm0800209,nm1921680,nm3053444,nm2757098,nm0317493,nm4160687,nm1293367,nm1411347  




In [14]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('imdb.db')

# Query and print the first few rows of each table
print(f"Showing first few rows of table title_principals:")
df = pd.read_sql_query(f"SELECT * from title_principals WHERE tconst  = 'tt1630029'", conn)
print(df)
print("\n")

# Close the connection
conn.close()


Showing first few rows of table title_principals:
      tconst  ordering     nconst  category            job    characters
0  tt1630029        10  nm0484457  producer       producer            \N
1  tt1630029         1  nm0941777     actor             \N      ["Jake"]
2  tt1630029         2  nm0757855   actress             \N   ["Neytiri"]
3  tt1630029         3  nm0000244   actress             \N      ["Kiri"]
4  tt1630029         4  nm0002332     actor             \N  ["Quaritch"]
5  tt1630029         5  nm0000116  director             \N            \N
6  tt1630029         6  nm0415425    writer  screenplay by            \N
7  tt1630029         7  nm0798646    writer  screenplay by            \N
8  tt1630029         8  nm0295264    writer       story by            \N
9  tt1630029         9  nm0004307    writer       story by            \N


